In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import urllib.request

base_url = "https://github.com/PrinciplesofRobotAutonomy/AA274A_midterm_FA25/raw/main/data/"
files = ["bunny_vertices.npy", "bunny_target.npy", "bunny_correspondences.npy", "bunny_is_inlier.npy", "line_2d_points.npy"]

for file in files:
    url = base_url + file
    urllib.request.urlretrieve(url, file)
    print(f"Downloaded {file}")

Downloaded bunny_vertices.npy
Downloaded bunny_target.npy
Downloaded bunny_correspondences.npy
Downloaded bunny_is_inlier.npy
Downloaded line_2d_points.npy


In [ ]:
# -------------------------------
# Fixed parameters (DO NOT TUNE)
# -------------------------------
SEED_LINE = 274
SEED_REG  = 274
EPSILON_INLIER = 0.08      # Part (a): line inlier threshold
N_ITER_LINE   = 150        # Part (a): RANSAC iterations

TAU_REG        = 0.002      # Part (c): registration inlier threshold (meters)
N_ITER_REG     = 200       # Part (c): RANSAC iterations
SAMPLE_SIZE    = 3         # Part (c): minimal sample siz

# Helper Functions

In [ ]:
def ensure_fig_dir():
    os.makedirs("figures", exist_ok=True)

def rmse(residuals: np.ndarray) -> float:
    """Root-mean-square of residuals (1D array)."""
    if residuals.size == 0:
        return np.inf
    return np.sqrt(np.mean(residuals**2))

def plot_line_ransac(points, m, b, inliers_mask, savepath):
    ensure_fig_dir()
    plt.figure(figsize=(6, 5))
    # raw points
    plt.scatter(points[:, 0], points[:, 1], s=10, c="#888888", label="All points")
    # inliers
    plt.scatter(points[inliers_mask, 0], points[inliers_mask, 1],
                s=12, c="#2ca02c", label="RANSAC inliers")

    # fitted line
    xs = np.linspace(points[:, 0].min(), points[:, 0].max(), 100)
    ys = m * xs + b
    plt.plot(xs, ys, lw=2, c="#d62728", label="Best-fit line")

    plt.title("RANSAC Line Fit")
    plt.xlabel("x"); plt.ylabel("y")
    plt.legend()
    plt.tight_layout()
    plt.savefig(savepath, dpi=150)
    plt.close()

def transform_points(P, R, t):
    """Apply rigid transform to points: P' = R P + t."""
    return (P @ R.T) + t

def registration_residuals(P, Q, R, t):
    """Return per-point Euclidean residuals ||R p + t - q||_2."""
    return np.linalg.norm(transform_points(P, R, t) - Q, axis=1)

def plot_bunny_before_after(P_src, Q_tgt, R, t, correspondences=None, inlier_mask=None, title="", savepath="figures/bunny_before_after.png"):
    """
    Plot 3D source/target point clouds before and after alignment.
    If correspondence and inlier masks are provided, only inlier correspondences are plotted.
    """
    ensure_fig_dir()
    P_aligned = (P_src @ R.T) + t

    # Apply correspondence indexing if given
    if correspondences is not None:
        P_corr = P_aligned[correspondences[:, 0]]
        Q_corr = Q_tgt[correspondences[:, 1]]
        if inlier_mask is not None:
            P_corr = P_corr[inlier_mask]
            Q_corr = Q_corr[inlier_mask]
    else:
        P_corr = P_aligned
        Q_corr = Q_tgt

    fig = plt.figure(figsize=(10,4.5))

    # Before alignment
    ax1 = fig.add_subplot(1,2,1, projection='3d')
    ax1.scatter(P_src[:,0], P_src[:,1], P_src[:,2], s=2, alpha=0.6, c='tab:blue', label="Source")
    ax1.scatter(Q_tgt[:,0], Q_tgt[:,1], Q_tgt[:,2], s=2, alpha=0.6, c='tab:orange', label="Target")
    ax1.set_title("Before Alignment"); ax1.legend()

    # After alignment — only inliers if provided
    ax2 = fig.add_subplot(1,2,2, projection='3d')
    ax2.scatter(P_corr[:,0], P_corr[:,1], P_corr[:,2], s=3, alpha=0.8, c='tab:green', label="Aligned Source (Inliers)")
    ax2.scatter(Q_corr[:,0], Q_corr[:,1], Q_corr[:,2], s=3, alpha=0.8, c='tab:red', label="Target (Inliers)")
    ax2.set_title("After Alignment (Inliers)"); ax2.legend()

    fig.suptitle(title)
    plt.tight_layout()
    plt.savefig(savepath, dpi=150)
    plt.close()

# Midterm Problems

In [ ]:
def ransac_line(points, eps=0.08, n_iter=150):
    """
    Robust line fitting y = m x + b using RANSAC.

    Args:
        points: (N,2) array of [x_i, y_i].
        eps:    inlier distance threshold (default 0.08).
        n_iter: number of RANSAC iterations.

    Returns:
        best_m, best_b, inliers_mask
    """
    N = points.shape[0]
    best_m, best_b = 0.0, 0.0
    best_inliers = np.zeros(N, dtype=bool)
    rng = np.random.default_rng(SEED_LINE)

    for k in range(n_iter):
        # --- Sample two distinct points ---
        idx = rng.choice(N, size=2, replace=False)
        (x_a, y_a), (x_b, y_b) = points[idx]

        # --- Fit candidate line y = m x + b ---
        m = (y_b - y_a) / (x_b - x_a)
        b = y_a - m * x_a

        # --- Compute distances and find inliers ---
        denom = np.hypot(m, 1.0)
        distances = np.abs(m * points[:, 0] - points[:, 1] + b) / denom
        inliers = distances <= eps
        if np.sum(inliers) > np.sum(best_inliers):
            best_inliers = inliers
            best_m, best_b = m, b

    # --- Refit line using least squares on all inliers ---
    if np.sum(best_inliers) >= 2:
        A = np.column_stack([points[best_inliers, 0], np.ones(np.sum(best_inliers))])
        y = points[best_inliers, 1]
        best_m, best_b = np.linalg.lstsq(A, y, rcond=None)[0]
    else:
        A = np.column_stack([points[:, 0], np.ones(N)])
        y = points[:, 1]
        best_m, best_b = np.linalg.lstsq(A, y, rcond=None)[0]
        best_inliers = np.ones(N, dtype=bool)

    return best_m, best_b, best_inliers

In [ ]:
def umeyama_alignment(P, Q):
    """
    Closed-form least-squares rigid alignment (Umeyama method).
    Args:
        P: (N,3) source points
        Q: (N,3) target points
    Returns:
        R (3,3), t (3,)
    """
    ### YOUR CODE HERE: compute centroids
    p_bar = P.mean(axis=0)
    q_bar = Q.mean(axis=0)

    ### YOUR CODE HERE: center the data
    P_centered = P - p_bar
    Q_centered = Q - q_bar

    ### YOUR CODE HERE: covariance and SVD
    H = P_centered.T @ Q_centered
    U, S, Vt = np.linalg.svd(H)

    ### YOUR CODE HERE: compute rotation and translation
    V = Vt.T
    D = np.eye(3)
    D[-1, -1] = np.linalg.det(V @ U.T)
    R = V @ D @ U.T
    t = q_bar - R @ p_bar

    return R, t


In [ ]:
def ransac_umeyama(P, Q, correspondences, tau=0.02, n_iter=200):
    """
    Robust Umeyama alignment using RANSAC.

    Args:
        P, Q: source and target point clouds (N,3)
        correspondences: (N,2) index pairs
        tau: inlier threshold
        n_iter: number of RANSAC iterations
    Returns:
        best_R, best_t, best_inliers
    """

    N = correspondences.shape[0]
    best_R = np.eye(3)
    best_t = np.zeros(3)
    best_inliers = np.zeros(N, dtype=bool)
    best_rmse = np.inf
    rng = np.random.default_rng(SEED_REG)

    P_corr = P[correspondences[:, 0]]
    Q_corr = Q[correspondences[:, 1]]

    for _ in range(n_iter):
        # === TODO(c1): randomly sample 3 correspondence pairs ===
        for _ in range(100):
            idx = rng.choice(N, size=SAMPLE_SIZE, replace=False)
            Ps = P_corr[idx]
            Qs = Q_corr[idx]
            area = np.linalg.norm(np.cross(Ps[1] - Ps[0], Ps[2] - Ps[0]))
            if area > 1e-6:
                break
        else:
            continue

        # === TODO(c2): estimate transform using Umeyama ===
        R_k, t_k = umeyama_alignment(Ps, Qs)

        # === TODO(c3): compute transformed distances for all correspondences ===
        P_all = (P_corr @ R_k.T) + t_k
        errors = np.linalg.norm(P_all - Q_corr, axis=1)
        inliers = errors < tau
        inlier_count = int(inliers.sum())
        if inlier_count > 0:
            rmse_k = np.sqrt(np.mean(errors[inliers]**2))
        else:
            rmse_k = np.inf

        best_count = int(best_inliers.sum())
        if (inlier_count > best_count) or (inlier_count == best_count and rmse_k < best_rmse):
            best_inliers = inliers
            best_R, best_t = R_k, t_k
            best_rmse = rmse_k

    # === TODO(c4): refit final (R, t) using inlier subset ===
    if best_inliers.any():
        P_best = P_corr[best_inliers]
        Q_best = Q_corr[best_inliers]
        best_R, best_t = umeyama_alignment(P_best, Q_best)
    else:
        best_R, best_t = umeyama_alignment(P_corr, Q_corr)

    return best_R, best_t, best_inliers


# Print Results for Each Method

In [ ]:
# Load data
line_path = "/content/line_2d_points.npy"
src_path  = "/content/bunny_vertices.npy"
tgt_path  = "/content/bunny_target.npy"

points_2d = np.load(line_path)            # (N, 2)
P_src = np.load(src_path)                 # (N, 3)
Q_tgt = np.load(tgt_path)                 # (N, 3)

# ---------------- (a) RANSAC line ----------------
m_hat, b_hat, inliers_mask = ransac_line(points_2d)
inlier_count = int(inliers_mask.sum())
print("=== (a) RANSAC line fitting ===")
print(f"  epsilon = {EPSILON_INLIER}, N_iter = {N_ITER_LINE}")
print(f"  Estimated (m, b): ({m_hat:.6f}, {b_hat:.6f})")
print(f"  Inliers: {inlier_count} / {points_2d.shape[0]}")
plot_line_ransac(points_2d, m_hat, b_hat, inliers_mask,
                  savepath="figures/line_ransac.png")
print("  Saved figure: figures/line_ransac.png\n")

# ---------------- (b) Umeyama alignment ---------
R_u, t_u = umeyama_alignment(P_src, Q_tgt)
res_u = registration_residuals(P_src, Q_tgt, R_u, t_u)
rmse_u = rmse(res_u)
print("=== (b) Umeyama (no scale) ===")
print("  R (rows):")
with np.printoptions(precision=6, suppress=True):
    print(R_u)
print("  t:", np.array2string(t_u, precision=6, suppress_small=True))
print(f"  RMSE (all pairs): {rmse_u:.6f}")
plot_bunny_before_after(P_src, Q_tgt, R_u, t_u,
    correspondences=np.load('/content/bunny_correspondences.npy'),
    inlier_mask=np.load('/content/bunny_is_inlier.npy'),
    title="Umeyama Alignment (Inlier Subset)",
    savepath="figures/bunny_before_after_umeyama.png")
# plot_bunny_before_after(P_src, Q_tgt, R_u, t_u,
#                         title="Umeyama (no scale): Before/After",
#                         savepath="figures/bunny_before_after_umeyama.png")
print("  Saved figure: figures/bunny_before_after_umeyama.png\n")

# ---------------- (c) REQUIRED: RANSAC registration ---------
R_r, t_r, inliers_r = ransac_umeyama(P_src, Q_tgt, np.load('/content/bunny_correspondences.npy'))
res_r_in = registration_residuals(P_src[inliers_r], Q_tgt[inliers_r], R_r, t_r) if inliers_r.any() else np.array([])
rmse_r_in = rmse(res_r_in)
print("=== (c) RANSAC registration (required) ===")
print(f"  tau = {TAU_REG}, N_iter = {N_ITER_REG}, sample size = {SAMPLE_SIZE}, seed = {SEED_REG}")
print(f"  Inliers found: {int(inliers_r.sum())} / {P_src.shape[0]}")
print("  R (rows):")
with np.printoptions(precision=6, suppress=True):
    print(R_r)
print("  t:", np.array2string(t_r, precision=6, suppress_small=True))
print(f"  RMSE on inliers: {rmse_r_in:.6f}")
plot_bunny_before_after(P_src, Q_tgt, R_r, t_r,
    correspondences=np.load('/content/bunny_correspondences.npy'),
    inlier_mask=np.load('/content/bunny_is_inlier.npy'),
    title="RANSAC Registration: Before/After",
    savepath="figures/bunny_before_after_ransac.png")
# plot_bunny_before_after(P_src, Q_tgt, R_r, t_r,
#                         title="RANSAC Registration: Before/After",
#                         savepath="figures/bunny_before_after_ransac.png")
print("  Saved figure: figures/bunny_before_after_ransac.png\n")

# -------------- brief comparison summary --------------
print("=== Summary comparison ===")
print(f"  Umeyama RMSE (all):      {rmse_u:.6f}")
print(f"  RANSAC RMSE (inliers):   {rmse_r_in:.6f}")
print(f"  RANSAC inliers count:    {int(inliers_r.sum())}")